In [1]:
import json

In [2]:
import requests

In [ ]:
import geopandas as gpd
import pandas as pd

## Open Geojson

In [4]:
latvia_url = f"https://gis.lgia.gov.lv/arcgis/rest/services/KP_OVERLAYS/Robezas/FeatureServer/9/query?where=1=1&outFields=*&f=geojson"
response = requests.get(latvia_url)

In [19]:
latvia_gdf = gpd.GeoDataFrame.from_features(response.json()["features"])
latvia_gdf.crs = 4326

In [20]:
latvia_gdf = latvia_gdf[["adreses_nosaukums", "geometry", "atvk"]]

## Get Open Population data

In [21]:
url = "https://data.stat.gov.lv:443/api/v1/en/OSP_PUB/START/POP/IR/IRE/IRE031"

In [22]:
data = {
    "query": [
        {"code": "ETHNICITY", "selection": {"filter": "item", "values": ["TOTAL"]}},
        {
            "code": "AREA",
            "selection": {
                "filter": "vs:VS_2021_AREA_LV_RPN_PLAN",
                "values": [
                    "LV",
                    "LV0001000",
                    "LV0002000",
                    "LV0003000",
                    "LV0031010",
                    "LV0004000",
                    "LV0005000",
                    "LV0040010",
                    "LV0006000",
                    "LV0054010",
                    "LV0007000",
                    "LV0020000",
                    "LV0021000",
                    "LV0022000",
                    "LV0023000",
                    "LV0024000",
                    "LV0025000",
                    "LV0026000",
                    "LV0027000",
                    "LV0028000",
                    "LV0029000",
                    "LV0030000",
                    "LV0031000",
                    "LV0032000",
                    "LV0033000",
                    "LV0034000",
                    "LV0035000",
                    "LV0036000",
                    "LV0037000",
                    "LV0038000",
                    "LV0039000",
                    "LV0040000",
                    "LV0041000",
                    "LV0042000",
                    "LV0043000",
                    "LV0044000",
                    "LV0045000",
                    "LV0046000",
                    "LV0047000",
                    "LV0048000",
                    "LV0049000",
                    "LV0051000",
                    "LV0052000",
                    "LV0053000",
                    "LV0054000",
                    "LV0055000",
                    "LV0056000",
                ],
            },
        },
        {"code": "ContentsCode", "selection": {"filter": "item", "values": ["IRE031"]}},
        {"code": "TIME", "selection": {"filter": "item", "values": ["2015", "2025"]}},
    ],
    "response": {"format": "JSON-stat"},
}

In [23]:
r = requests.post(url, data=json.dumps(data))

In [24]:
json_data = r.json()
indexes = json_data["dataset"]["dimension"]["AREA"]["category"]["index"]
labels = json_data["dataset"]["dimension"]["AREA"]["category"]["label"]
data = json_data["dataset"]["value"]

In [25]:
data_2015 = data[0::2]
data_2025 = data[1::2]

In [26]:
data_list = [
    {"name": labels[key], "atvk": key, "2015": data_2015[indx], "2025": data_2025[indx]}
    for key, indx in indexes.items()
]

In [27]:
population_df = pd.DataFrame(data_list)
population_df["atvk"] = population_df["atvk"].str[2:]

## Merge data and export

In [28]:
latvia_population_gdf = latvia_gdf.merge(population_df, on="atvk", how="left")

In [29]:
latvia_population_gdf["change"] = (
    latvia_population_gdf["2025"] - latvia_population_gdf["2015"]
)
latvia_population_gdf["rate"] = (
    (latvia_population_gdf["change"] / latvia_population_gdf["2015"]) * 100
).round(decimals=2)

In [30]:
latvia_population_gdf = latvia_population_gdf.to_crs(3059)
latvia_population_gdf.to_file("data/latvian_population.gpkg")